#자연어처리 중간고사 연습한 기록입니다.
#아래 연구들(머신러닝, 딥러닝) 결과 가장 좋은 성능이 나온 코드를 따로 정리한 파일은 "2021_NLP_mid"파일입니다.
#정주호 / 2019312072 / 78.6%

In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
import pandas as pd

train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

#Data Preprocessing 

In [ ]:
# 한글만 남기기 위해 정규표현식 사용 
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
train_df[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 특수문자들을 제거하였는데, 리뷰가 특수문자로만 이루어져 있는 document는 그 과정에서 null값만을 갖게 되어서 삭제해야 한다. 
import numpy as np
train_df['document'] = train_df['document'].str.replace('^ +', "") #특수문자를 공백으로 바꾸기 
train_df['document'].replace('', np.nan, inplace=True)
train_df = train_df.dropna(how = 'any')

In [ ]:
#test data set에도 동일하게 전처리를 실행 
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ") # 정규 표현식 수행
test_df['document'] = test_df['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_df = test_df.dropna(how='any') # Null 값 제거

#훈련/테스트셋 구축 및 토큰화

In [ ]:
train_text = train_df['document'][:8000]
test_text = test_df['document'][8000:9000]
train_label = train_df['label'][:8000]
test_label = test_df['label'][8000:9000]

In [ ]:
train_text[0]
vectorizer = TfidfVectorizer(max_features=20000)
vectors = vectorizer.fit_transform(train_text)

'아 더빙   진짜 짜증나네요 목소리'

In [ ]:
test_text[:3]

8089          색깔없는 영화  
8090       오빠들사랑해요 진심으로
8091    조승우 연기는 부들부들이었다
Name: document, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_text)

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
vectors_test = vectorizer.transform(test_text)
clf = MultinomialNB(alpha=.01)
clf.fit(vectors, train_label)
pred = clf.predict(vectors_test)
metrics.f1_score(test_label, pred, average='macro')

0.7325504172513997

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
}  
grid_search= GridSearchCV(clf, parameters)
grid_search.fit(vectors, train_label)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=0.01, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 1e-05)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
grid_search.best_estimator_

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [ ]:
pred = grid_search.predict(vectors_test)

metrics.f1_score(test_label,pred,average='macro')

0.7435897435897436

In [ ]:
random_seed=2020

def model_selection(X_train, y_train, kfold=3):
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from xgboost import XGBClassifier
    from sklearn.model_selection import cross_val_score
    from tqdm.notebook import tqdm
    import pandas as pd

    models = [
        LogisticRegression(),
        XGBClassifier(random_state=random_seed),
        SVC(), DecisionTreeClassifier(),MultinomialNB()
    ]
    
    progress_bar = tqdm(total = len(models) * kfold)
    
    entries = []
    for model in models:
        model_name = model.__class__.__name__
        accuracies = cross_val_score(model, X_train, y_train, 
                        scoring='accuracy', cv=kfold, n_jobs=-1)
    
        for fold_idx, accuracy in enumerate(accuracies):
            progress_bar.update()
            entries.append((model_name, fold_idx, accuracy))
    
    progress_bar.close()
    
    cv_result = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'], index=range(kfold * len(models)))
    return cv_result

In [ ]:
model_selection_result = model_selection(vectors, train_label)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
model_selection_result

,model_name,fold_idx,accuracy
0,LogisticRegression,0,0.765654
1,LogisticRegression,1,0.780652
2,LogisticRegression,2,0.765191
3,XGBClassifier,0,0.667417
4,XGBClassifier,1,0.664042
5,XGBClassifier,2,0.651538
6,SVC,0,0.762655
7,SVC,1,0.784777
8,SVC,2,0.769317
9,DecisionTreeClassifier,0,0.655043


In [ ]:
pd.pivot_table(model_selection_result, values='accuracy', index=['model_name'], aggfunc=np.mean, fill_value=0)
#각 모델들의 정확도를 평균내서 비교함
#NB모델이 가장 성능이 높다. 

,accuracy
model_name,
DecisionTreeClassifier,0.670377
LogisticRegression,0.770499
MultinomialNB,0.784000
SVC,0.772250
XGBClassifier,0.660999


In [ ]:
clf = MultinomialNB(alpha=0.6)
clf.fit(vectors, train_label)
pred = clf.predict(vectors_test)
metrics.f1_score(test_label, pred, average='macro')

0.7455511522325382

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 파라메터 후보
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

# 그리드 서치 진행
grid_search = GridSearchCV( LogisticRegression(), param_grid, cv=5)        

grid_search.fit(vectors, train_label)
grid_search.score(vectors_test, test_label)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
             'gamma': [0.001, 0.01, 0.1, 1, 10, 100] }
# GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)

grid_search.fit(vectors, train_label)
grid_search.score(vectors_test, test_label)

0.771

#ML-Classification을 이용하였을 시 MultinomialNB 모델의 성능이 가장 좋았고, 정확도를 높이기 위해 하이퍼 파라미터를 찾은 결과 정확도는 75%





#한국어 전처리 패키지 중 py-hanspell이라는 네이버 한글 맞춤법 검사기를 바탕으로 만들어진 패키지가 존재. 이를 바탕으로 전체 train, test data set을 전처리한 후 정확도를 비교해봄 

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-jma3xdel
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-jma3xdel
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=1965c505c8044bd7b473fefc72196606cdfe1142cfcc3f4eed2bd3057d7c7e96
  Stored in directory: /tmp/pip-ephem-wheel-cache-5irvzeib/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [ ]:
train_text

0                                     아 더빙   진짜 짜증나네요 목소리
1                       흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나
2                                       너무재밓었다그래서보는것을추천한다
3                           교도소 이야기구먼   솔직히 재미는 없다  평점 조정
4       사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                              ...                        
8051    어릴때봤던영화  너무좋은기억으로남아있어서 최근에 다시다운받아봤다 나   어릴땐어떤아...
8052                                   신이의 변하기 전 모습이 그립다 
8053                                  여성의 각성과 가부장적 권위의 몰락
8054                                    살 조카랑 같이 자고나왔어요ㅠㅋ
8055                                      오현경 한번나와줘야 쩔지 캬
Name: document, Length: 8000, dtype: object

한번에 전처리 하는 방법을 몰라서 8000개의 데이터를 모두 읽어와서 한 리뷰별로 전처리 후 저장함. 이때 시간이 많이 걸리므로 colab에서 제공하는 GPU를 빌려와서 사용함. 

In [ ]:
#교수님 평가하실때 시간 단축을 위해 hanspell로 전처리한 train, test data set csv파일을 따로 제출하였습니다. 
#train_text : preprocessed_train_data
#test_text : preprocessed_test_data
x_train = pd.read_csv("preprocessed_train_data.csv", index_col = 0)
x_test = pd.read_csv("preprocessed_test_data.csv", index_col = 0)

In [ ]:
#데이터 전처리과정, csv파일 이용시 실행시키지 않아도 되는 셀 
from tqdm import tqdm

x_train=[]
for sen in tqdm(train_text):
  spelled_sent = spell_checker.check(sen)
  hanspell_sent = spelled_sent.checked
  x_train.append(hanspell_sent)

x_train = pd.DataFrame(x_train)

100%|██████████| 8000/8000 [12:24<00:00, 10.74it/s]


In [ ]:
x_train['0']

0                                    아 더빙   진짜 짜증 나네요 목소리
1                  흠   포스터 보고 초등학생영화 줄    오버 연기조차 가볍지 않구나
2                                       너무재밓었다그래서보는것을추천한다
3                           교도소 이야기구먼   솔직히 재미는 없다  평점 조정
4       사이먼 페그의 익살스러운 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커...
                              ...                        
7995    어릴 때 봤던 영화  너무 좋은 기억으로 남아있어서 최근에 다시 다운로드해봤다 나어...
7996                                   신이의 변하기 전 모습이 그립다 
7997                                  여성의 각성과 가부장적 권위의 몰락
7998                                    살 조카랑 같이 자고나왔어요ㅠㅋ
7999                                     오현경 한번 나와줘야 쩔지 캬
Name: 0, Length: 8000, dtype: object

In [ ]:
from tqdm import tqdm

x_test=[]
for sen in tqdm(test_text):
  spelled_sent = spell_checker.check(sen)
  hanspell_sent = spelled_sent.checked
  x_test.append(hanspell_sent)

x_test = pd.DataFrame(x_test)

100%|██████████| 1000/1000 [01:41<00:00,  9.88it/s]


In [ ]:
x_test

,0
0,색깔 없는 영화
1,오빠들 사랑해요 진심으로
2,조승우 연기는 부들부들이었다
3,제발 이제 캐릭터랑 맞지도 않는 사람 가져다가 쓰지 마세요 이러니까 대한민국 성우가...
4,평범하지 않은 사랑을 노래 안에 녹이다
...,...
995,반전은 개뿔 그냥 그런데 평점이 너무 높아서 낮춰주러 왔음 ㅡㅡ
996,감독은 한국 관객들이 이 영화를 이해하지 못한다고 생떼 부린다 일본으로 꺼져주세요
997,어렸을 때 봤을 때도 지루하고 재미없던 영화 그렇게 칭찬이 많던 영화인데 그 칭찬은...
998,베라에게 총이 없었다고 해도 언젠가는 일어날 일이었다


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(x_train['0'])

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
vectors_test = vectorizer.transform(x_test['0'])
clf = MultinomialNB(alpha=.01)
clf.fit(vectors, train_label)
pred = clf.predict(vectors_test)
metrics.f1_score(test_label, pred, average='macro')

0.7559689688935753

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
}  
grid_search= GridSearchCV(clf, parameters)
grid_search.fit(vectors, train_label)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=0.01, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 1e-05)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
grid_search.best_estimator_

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [ ]:
pred = grid_search.predict(vectors_test)

metrics.f1_score(test_label,pred,average='macro')

0.786096256684492

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)로 분류할 시 78.6%로 가장 정확도가 높음 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(x_train['0'])

vectors_test = vectorizer.transform(x_test['0'])
clf = LogisticRegression()
clf.fit(vectors, train_label)
pred = clf.predict(vectors_test)
metrics.f1_score(test_label, pred, average='macro')

0.781650641025641

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(x_train['0'])

vectors_test = vectorizer.transform(x_test['0'])
clf = SVC()
clf.fit(vectors, train_label)
pred = clf.predict(vectors_test)
metrics.f1_score(test_label, pred, average='macro')

0.7821352949857986

#py-Hanspell로 전처리 한 결과(한국어 맞춤법 및 띄어쓰기 전처리 ) 78%의 정확도를 나타냄(정확도가 5%이상 올라간 것을 확인할 수 있음)

In [ ]:
x_train.to_csv("preprocessed_train_data.csv")
x_test.to_csv("preprocessed_test_data.csv")

#DL-classification : 딥러닝 모델을 이용하여 분류해보기 

In [ ]:
!pip install konlpy 

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 448 kB 49.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
okt = Okt()
train_text = []
test_text = []
for text in train_df['document'][:8000]:
  #train_text.append(word_tokenize(text))
  train_text.append(okt.morphs(text, stem=True))
for text in test_df['document'][8000:9000]:
  #test_text.append(word_tokenize(text))
  test_text.append(okt.morphs(text, stem=True))
train_label = train_df['label'][:8000]
test_label = test_df['label'][8000:9000]

In [ ]:
train_text[:5]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스럽다',
  '연기',
  '가',
  '돋보이다',
  '영화',
  '스파이더맨',
  '에서',
  '늙다',
  '보이다',
  '하다',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이쁘다',
  '보이다']]

In [ ]:
train_text[0]

['아', '더빙', '진짜', '짜증나다', '목소리']

In [ ]:
test_text[:5]

[['색깔', '없다', '영화'],
 ['오빠', '들', '사랑', '해', '요', '진심', '으로'],
 ['조승우', '연기', '는', '부들부들', '이다'],
 ['제발',
  '이제',
  '캐릭터',
  '랑',
  '맞다',
  '않다',
  '사람',
  '가다',
  '쓰다',
  '말다',
  '이러니까',
  '대한민국',
  '성우',
  '가',
  '있다',
  '자리',
  '가',
  '없다'],
 ['평범하다', '않다', '사랑', '을', '노래', '안', '에', '녹', '이다']]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)

In [ ]:
print(tokenizer.word_index)

{'영화': 1, '보다': 2, '을': 3, '없다': 4, '이다': 5, '있다': 6, '좋다': 7, '다': 8, '정말': 9, '너무': 10, '재밌다': 11, '만': 12, '같다': 13, '되다': 14, '적': 15, '점': 16, '진짜': 17, '아니다': 18, '로': 19, '에서': 20, '않다': 21, '나오다': 22, '평점': 23, '연기': 24, '만들다': 25, '것': 26, '최고': 27, '나': 28, '안': 29, '인': 30, '내': 31, '못': 32, '사람': 33, '그': 34, '스토리': 35, '드라마': 36, '왜': 37, '아깝다': 38, '게': 39, '감동': 40, '보고': 41, '고': 42, '때': 43, '생각': 44, '이렇다': 45, '말': 46, '감독': 47, 'ㅋㅋ': 48, '그냥': 49, '시간': 50, '아': 51, '내용': 52, '요': 53, '재미있다': 54, '거': 55, '배우': 56, '가다': 57, '더': 58, '재미': 59, '재미없다': 60, '지루하다': 61, '하고': 62, '자다': 63, '주다': 64, '중': 65, '쓰레기': 66, '모르다': 67, '까지': 68, '뭐': 69, '작품': 70, '네': 71, '들다': 72, '알다': 73, '수': 74, '그렇다': 75, '사랑': 76, '싶다': 77, '하나': 78, '지': 79, '볼': 80, '개': 81, '이건': 82, '돈': 83, '정도': 84, '저': 85, '마지막': 86, '다시': 87, '주인공': 88, '이렇게': 89, '액션': 90, '기': 91, 'ㅋ': 92, '차다': 93, '걸': 94, '연출': 95, '오다': 96, '느낌': 97, 'ㅋㅋㅋ': 98, '나다': 99, '최악': 100, '많다': 101, '완전': 102

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(train_text)
X_train = tokenizer.texts_to_sequences(train_text)
X_test = tokenizer.texts_to_sequences(test_text)

#수업시간에 배운 DL-classification을 이용하여 분류해봄. shape을 동일하게 하기 위해 feature 개수는 25000개로 한정  

In [ ]:
vectorizer = TfidfVectorizer(max_features=29000)
vectors = vectorizer.fit_transform(train_text)
vectors_test = vectorizer.transform(test_text)

NameError: ignored

In [ ]:
x_train = vectors.toarray()
x_test = vectors_test.toarray()

NameError: ignored

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(8000, 25245)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]
history = model.fit(partial_x_train,partial_y_train,epochs=20,batch_size=512,validation_data=(x_val, y_val))

Epoch 1/20
14/14 [==============================] - 1s 58ms/step - loss: 0.6881 - accuracy: 0.6029 - val_loss: 0.6802 - val_accuracy: 0.7340
Epoch 2/20
14/14 [==============================] - 0s 33ms/step - loss: 0.6590 - accuracy: 0.8584 - val_loss: 0.6618 - val_accuracy: 0.7350
Epoch 3/20
14/14 [==============================] - 0s 34ms/step - loss: 0.6199 - accuracy: 0.9039 - val_loss: 0.6410 - val_accuracy: 0.7650
Epoch 4/20
14/14 [==============================] - 0s 34ms/step - loss: 0.5745 - accuracy: 0.9340 - val_loss: 0.6186 - val_accuracy: 0.7760
Epoch 5/20
14/14 [==============================] - 0s 33ms/step - loss: 0.5250 - accuracy: 0.9499 - val_loss: 0.5960 - val_accuracy: 0.7710
Epoch 6/20
14/14 [==============================] - 0s 32ms/step - loss: 0.4735 - accuracy: 0.9601 - val_loss: 0.5731 - val_accuracy: 0.7750
Epoch 7/20
14/14 [==============================] - 0s 32ms/step - loss: 0.4216 - accuracy: 0.9657 - val_loss: 0.5512 - val_accuracy: 0.7730
Epoch 8/20
14

In [ ]:
results = model.evaluate(x_test,y_test)

32/32 [==============================] - 0s 4ms/step - loss: 0.5298 - accuracy: 0.7530


#딥러닝 분류 결과 : 주어진 train set에 대해선 99%의 정확도를 보인다. 그러나 test set에 대하여 74.5%의 낮은 정확도를 보이는 결과, 과적합이 일어났다고 판단할 수 있다. 